<a href="https://colab.research.google.com/github/apalibaeva/simulative-python-base/blob/main/%D0%93%D0%BB%D0%B0%D0%B2%D0%B0_10_%D0%92%D1%82%D0%BE%D1%80%D0%BE%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [3]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2024-10-18 12:01:07--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2024-10-18 12:01:07--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2024-10-18 12:01:07 (33.8 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Как примерно должен выглядеть результат:

In [4]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [5]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев**.

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023).

**Решение**

Напишите свое решение ниже

In [17]:
import gspread, pprint
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

def generate_report(sheet1, sheet2, sheet3):
  # Указываем необходимые права доступа к таблицам
  scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

  # Загружаем ключи аутентификации из файла json
  creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

  # Авторизуемся в Google Sheets API
  client = gspread.authorize(creds)

  #получаем информацию с каждого листа
  sheet1_open = client.open("Installments").worksheet(sheet1)
  sheet2_open = client.open("Installments").worksheet(sheet2)
  sheet3_open = client.open("Installments").worksheet(sheet3)

  sheet1_data = sheet1_open.get_all_records()
  sheet2_data = sheet2_open.get_all_records()
  sheet3_data = sheet3_open.get_all_records()

  #отбираем только тех студентов, у кого оформлена рассрочка(из первого листа)
  students_info = []

  for row in sheet1_data:
    if row['installment'] == 'Y':
      students_info.append(row)

  #считаем долг по каждому студенту
  students_debts = []

  for r1, r2, r3 in zip(students_info, sheet2_data, sheet3_data):
    exp_date = datetime.strptime(r2['expected_payment_date'], '%d.%m.%Y')
    last_day = datetime.strptime('01.03.2023', '%d.%m.%Y')
    if exp_date <= last_day: #если дата следующей оплаты еще не наступила на 01.03.2024
      payments_cnt = (last_day - exp_date).days // 183 + 1 #считаю сколько в общем платежей должно было быть у студента
      debt = r3['one-time_payment'] * payments_cnt if r3['one-time_payment'] * payments_cnt < r3['left_to_pay'] else r3['left_to_pay']
      students_debts.append({'student_id': r1['student_id'], 'debt': debt, 'student_name': r1['student_name']})
      print({'student_id': r1['student_id'], 'debt': debt, 'student_name': r1['student_name']})

  #далее записываем полученную информацию в массив в нужном формате
  check = []
  for el in students_debts:
    if el['debt'] > 0:
      name = el['student_name']
      debt = el['debt']
      if students_debts.index(el) == len(students_debts) - 1:
        check.append(f'Студент {name} - долг {debt} рублей')
      else:
        check.append(f'Студент {name} - долг {debt} рублей\n')

  #записываем информацию в текстовый файл
  with open('student_debt_report.txt', 'w') as f:
    for i in check:
      f.write(i)

In [18]:
generate_report('Лист1', 'Лист2', 'Лист3')

{'student_id': 2, 'debt': 266666, 'student_name': 'Кузнецова К.А.'}
{'student_id': 3, 'debt': 66666, 'student_name': 'Петров К.А.'}
{'student_id': 6, 'debt': 100000, 'student_name': 'Иванов А.А.'}
{'student_id': 10, 'debt': 116666, 'student_name': 'Иванов П.П.'}
{'student_id': 12, 'debt': 116666, 'student_name': 'Петров И.И.'}
{'student_id': 13, 'debt': 16666, 'student_name': 'Петров А.А.'}
{'student_id': 14, 'debt': 41666, 'student_name': 'Иванов Е.В.'}
{'student_id': 15, 'debt': 100000, 'student_name': 'Николаева Е.В.'}
{'student_id': 18, 'debt': 233333, 'student_name': 'Смирнова Е.В.'}
{'student_id': 21, 'debt': 33332, 'student_name': 'Николаева М.А.'}
{'student_id': 22, 'debt': 133332, 'student_name': 'Кузнецова А.А.'}
{'student_id': 23, 'debt': 25000, 'student_name': 'Иванов А.А.'}
{'student_id': 25, 'debt': 133332, 'student_name': 'Петров И.И.'}
{'student_id': 28, 'debt': 133333, 'student_name': 'Кузнецова К.А.'}
{'student_id': 30, 'debt': 66666, 'student_name': 'Кузнецова Е.В.'}

In [19]:
with open('student_debt_report.txt', 'r') as f:
  data1 = f.readlines()


with open('student_debt.txt', 'r') as f:
  data2 = f.readlines()


In [20]:
for i, j in zip(data1, data2):
  if i != j:
    print('My: ', i, '\nTheir: ', j)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [21]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2024-10-18 12:04:00--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2024-10-18 12:04:00--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt.1’

student_debt.txt.1  100%[===================>]  11.06K  --.-KB/s    in 0s      

2024-10-18 12:04:01 (72.1 MB/

In [22]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
